In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [1]:
!pip install opendatasets --upgrade --quiet
import opendatasets as od
import os
import tensorflow as tf

In [3]:
dataset_url = 'https://www.kaggle.com/virtualdvid/oregon-wildlife'
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: wastex
Your Kaggle Key: ··········


  0%|          | 0.00/4.55G [00:00<?, ?B/s]

100%|██████████| 4.55G/4.55G [01:30<00:00, 54.1MB/s]

In [4]:
data_dir = './oregon-wildlife/oregon_wildlife/oregon_wildlife'
print(os.listdir(data_dir))

['columbian_black-tailed_deer', 'bald_eagle', 'coyote', 'cougar', 'black_bear', 'elk', 'sea_lions', 'raven', 'seals', 'bobcat', 'gray_fox', 'red_fox', 'raccoon', 'canada_lynx', 'mountain_beaver', 'deer', 'nutria', 'ringtail', 'gray_wolf', 'virginia_opossum']


In [5]:
os.makedirs('./logs')

In [6]:
def clear_data(root):
    for folder_name in os.listdir(root):
        for file_name in os.listdir(root+'/'+folder_name):
            base_name, ext = os.path.splitext(file_name)
            if ext != '.jpg':
                os.remove(root+'/'+folder_name+'/'+file_name)
clear_data(data_dir)

In [7]:
log_dir = './logs'
batch_size = 128
img_size = 224
num_classes = 20

In [8]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(data_dir, labels='inferred',
            color_mode='rgb', batch_size=batch_size, image_size=(img_size, img_size),
            shuffle=True, seed=41, validation_split=0.3, subset='training')
valid_ds = tf.keras.preprocessing.image_dataset_from_directory(data_dir, labels='inferred',
            color_mode='rgb', batch_size=batch_size, image_size=(img_size, img_size),
            shuffle=True, seed=41, validation_split=0.3, subset='validation')

Found 12786 files belonging to 20 classes.
Using 8951 files for training.
Found 12786 files belonging to 20 classes.
Using 3835 files for validation.


In [9]:
def input_preprocess(image, label):
    label = tf.one_hot(label, num_classes)
    return image, label


train_ds = train_ds.map(
    input_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE
)
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

valid_ds = valid_ds.map(input_preprocess)

In [10]:
def build_model():
    inputs = tf.keras.Input(shape=(img_size, img_size, 3))
    model = tf.keras.applications.EfficientNetB0(input_tensor=inputs, include_top=False, weights="imagenet")
    model.trainable = False
    x = tf.keras.layers.GlobalAveragePooling2D()(model.output)
    outputs = tf.keras.layers.Dense(num_classes, activation=tf.keras.activations.softmax)(x)
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    model.compile(
        optimizer=tf.optimizers.Adam(0.001),
        loss=tf.keras.losses.categorical_crossentropy,
        metrics=[tf.keras.metrics.categorical_accuracy],
    )
    return model

In [11]:
model = build_model()
    
model.fit(
    train_ds,
    epochs=50,
    validation_data=valid_ds,
    callbacks=[
        tf.keras.callbacks.TensorBoard(log_dir),
    ]
  )

16711680/16705208 [==============================] - 0s 0us/step
Epoch 1/50
70/70 [==============================] - 207s 2s/step - loss: 1.9366 - categorical_accuracy: 0.5076 - val_loss: 0.6129 - val_categorical_accuracy: 0.8412
Epoch 2/50
70/70 [==============================] - 173s 2s/step - loss: 0.5794 - categorical_accuracy: 0.8506 - val_loss: 0.4553 - val_categorical_accuracy: 0.8683
Epoch 3/50
70/70 [==============================] - 141s 2s/step - loss: 0.4399 - categorical_accuracy: 0.8756 - val_loss: 0.3943 - val_categorical_accuracy: 0.8855
Epoch 4/50
70/70 [==============================] - 131s 2s/step - loss: 0.3753 - categorical_accuracy: 0.8991 - val_loss: 0.3585 - val_categorical_accuracy: 0.8941
Epoch 5/50
70/70 [==============================] - 132s 2s/step - loss: 0.3261 - categorical_accuracy: 0.9077 - val_loss: 0.3337 - val_categorical_accuracy: 0.8993
Epoch 6/50
70/70 [==============================] - 132s 2s/step - loss: 0.3019 - categorical_accuracy: 0.9149